In [19]:
import time
import torch
import mujoco
import mujoco.viewer
import cv2
import os
import numpy as np
from scipy.spatial.transform import Rotation as R, Slerp
import yaml
import random
import re
import mlflow
import gymnasium as gym
from gymnasium import spaces
import torch.nn as nn
from stable_baselines3 import PPO
from stable_baselines3.common.evaluation import evaluate_policy
from stable_baselines3.common.env_checker import check_env
from stable_baselines3.common.callbacks import BaseCallback
from sb3_contrib import RecurrentPPO
import json
from collections import deque

# Configuración de Telegram (opcional)
TELEGRAM_TOKEN = '7979041854:AAE1zqPs6nNvR-CewP2rRx0_T_enAcmjFEE'
TELEGRAM_CHAT_ID = '1446668173'

def send_text(message, token=TELEGRAM_TOKEN, chat_id=TELEGRAM_CHAT_ID):
    """Función para enviar mensajes por Telegram"""
    try:
        import requests
        url = f"https://api.telegram.org/bot{token}/sendMessage"
        data = {"chat_id": chat_id, "text": message}
        requests.post(url, data=data)
    except:
        print(f"Telegram message: {message}")

def send_video(video_path, caption="", token=TELEGRAM_TOKEN, chat_id=TELEGRAM_CHAT_ID):
    """Función para enviar videos por Telegram"""
    try:
        import requests
        url = f"https://api.telegram.org/bot{token}/sendVideo"
        with open(video_path, 'rb') as video:
            files = {'video': video}
            data = {'chat_id': chat_id, 'caption': caption}
            requests.post(url, files=files, data=data)
    except Exception as e:
        print(f"Could not send video: {e}")

def record_final_video(model, env, video_path="final_agent.mp4", max_steps=200, fps=20, log_mlflow=True):
    """Record a video of the trained agent using the panoramic view"""
    obs, info = env.reset()
    imgs = env.render()
    
    if isinstance(imgs, tuple) and len(imgs) >= 2:
        frame = imgs[1]  # panoramic_view_bgr
    else:
        frame = imgs

    height, width, _ = frame.shape
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    out = cv2.VideoWriter(video_path, fourcc, fps, (width, height))

    for step in range(max_steps):
        action, _ = model.predict(obs, deterministic=True)
        obs, reward, done, truncated, info = env.step(action)

        imgs = env.render()
        if isinstance(imgs, tuple) and len(imgs) >= 2:
            frame = imgs[1]
        else:
            frame = imgs
            
        out.write(frame)
        if done:
            print(f"[{env.objeto}] Episode finished at step {step}, reward: {reward}")
            break

    out.release()
    print(f"[{env.objeto}] Video saved to {video_path}")

    if log_mlflow:
        try:
            mlflow.log_artifact(video_path)
            print(f"[{env.objeto}] Video logged to MLflow: {video_path}")
        except Exception as e:
            print(f"Could not log video to MLflow: {e}")

def get_camera_image(model, data, cam_name, width=400, height=400, options=None):
    """Get camera image from Mujoco"""
    renderer = mujoco.Renderer(model, height=height, width=width)
    if options is None:
        renderer.update_scene(data, camera=cam_name)
    else:
        renderer.update_scene(data, cam_name, options)
    rgb_array = renderer.render()
    return rgb_array

def extract_s_pattern(input_string):
    """Extract scene pattern from filename"""
    pattern = r's\d+'
    match = re.search(pattern, input_string)
    if match:
        s_pattern = match.group()
        before = input_string[:match.start()]
        after = input_string[match.end():]
        return before[:-1], s_pattern, after
    else:
        return None, None, None

class TrainableObjectEmbeddingManager(nn.Module):
    """Object embeddings that can be trained during RL"""
    
    def __init__(self, embedding_dim=16):
        super().__init__()
        self.embedding_dim = embedding_dim
        self.known_objects = self._discover_all_objects()
        
        # Create trainable embedding layer
        self.embedding_layer = nn.Embedding(
            num_embeddings=len(self.known_objects) + 1,  # +1 for unknown objects
            embedding_dim=embedding_dim,
            padding_idx=0  # Use index 0 for unknown objects
        )
        
        # Initialize with reasonable values
        self._initialize_embeddings()
        
        # Mapping from object names to indices
        self.object_to_idx = {obj: idx+1 for idx, obj in enumerate(self.known_objects)}
        self.object_to_idx['unknown'] = 0  # Default for unknown objects
        
        print(f"Initialized trainable embeddings for {len(self.known_objects)} objects")
        print(f"Objects: {self.known_objects}")
        
    def _discover_all_objects(self):
        """Discover all object names from scene files"""
        scenes_dir = "/home/diego/TFM/models/scenes/"
        all_scenes = os.listdir(scenes_dir)
        object_names = set()
        
        for scene_file in all_scenes:
            if scene_file.endswith('_scene.xml'):
                parts = scene_file.split('_')
                if len(parts) >= 2:
                    object_names.add(parts[0])
        
        return sorted(object_names)
    
    def _initialize_embeddings(self):
        """Initialize embeddings with geometric priors"""
        with torch.no_grad():
            # Initialize all embeddings with small random values first
            nn.init.normal_(self.embedding_layer.weight, mean=0.0, std=0.1)
            
            # Geometric priors based on object names
            for idx, obj_name in enumerate(self.known_objects):
                obj_lower = obj_name.lower()
                
                if 'cube' in obj_lower:
                    # Cubes: symmetric, flat surfaces
                    init_embedding = torch.tensor([1.0, 0.0, 0.0, 0.0] + [0.1] * (self.embedding_dim - 4))
                elif 'sphere' in obj_lower:
                    # Spheres: perfectly symmetric
                    init_embedding = torch.tensor([0.0, 1.0, 0.0, 0.0] + [0.1] * (self.embedding_dim - 4))
                elif 'cylinder' in obj_lower:
                    # Cylinders: axial symmetry
                    init_embedding = torch.tensor([0.0, 0.0, 1.0, 0.0] + [0.1] * (self.embedding_dim - 4))
                elif 'pyramid' in obj_lower:
                    # Pyramids: pointed
                    init_embedding = torch.tensor([0.0, 0.0, 0.0, 1.0] + [0.1] * (self.embedding_dim - 4))
                elif 'torus' in obj_lower:
                    # Torus: donut shape
                    init_embedding = torch.tensor([0.5, 0.5, 0.0, 0.0] + [0.1] * (self.embedding_dim - 4))
                elif any(x in obj_lower for x in ['apple', 'banana']):
                    # Fruits: organic shapes
                    init_embedding = torch.tensor([0.2, 0.7, 0.1, 0.0] + [0.1] * (self.embedding_dim - 4))
                elif any(x in obj_lower for x in ['bowl', 'cup', 'mug', 'wineglass']):
                    # Kitchen items: curved containers
                    init_embedding = torch.tensor([0.3, 0.6, 0.1, 0.0] + [0.1] * (self.embedding_dim - 4))
                elif any(x in obj_lower for x in ['camera', 'phone', 'flashlight']):
                    # Electronics: rectangular with details
                    init_embedding = torch.tensor([0.8, 0.1, 0.1, 0.0] + [0.1] * (self.embedding_dim - 4))
                elif any(x in obj_lower for x in ['hammer', 'scissors', 'knife']):
                    # Tools: elongated
                    init_embedding = torch.tensor([0.7, 0.2, 0.1, 0.0] + [0.1] * (self.embedding_dim - 4))
                else:
                    # Default: mixed properties (keep the random initialization)
                    continue
                
                # Apply the geometric prior
                self.embedding_layer.weight.data[idx+1] = init_embedding
    
    def get_embedding(self, object_name, as_tensor=False):
        """Get embedding for object - returns tensor for training"""
        obj_idx = self.object_to_idx.get(object_name, 0)  # 0 for unknown
        embedding = self.embedding_layer(torch.tensor([obj_idx]))
        
        if as_tensor:
            return embedding.squeeze(0)  # Return as tensor for training
        else:
            return embedding.squeeze(0).detach().cpu().numpy()  # Return as numpy for env
    
    def get_all_embeddings(self):
        """Get all embeddings for analysis"""
        return self.embedding_layer.weight.detach().cpu().numpy()
    
    def visualize_embeddings(self, step=0):
        """Visualize learned embeddings"""
        try:
            import matplotlib.pyplot as plt
            from sklearn.manifold import TSNE
            
            embeddings = self.get_all_embeddings()
            object_names = ['unknown'] + self.known_objects
            
            # Use T-SNE for dimensionality reduction
            tsne = TSNE(n_components=2, random_state=42, perplexity=min(5, len(embeddings)-1))
            embeddings_2d = tsne.fit_transform(embeddings)
            
            plt.figure(figsize=(12, 10))
            scatter = plt.scatter(embeddings_2d[:, 0], embeddings_2d[:, 1], alpha=0.7, s=100)
            
            # Add labels
            for i, obj in enumerate(object_names):
                plt.annotate(obj, (embeddings_2d[i, 0], embeddings_2d[i, 1]), 
                           xytext=(5, 5), textcoords='offset points', fontsize=8)
            
            plt.title(f'Learned Object Embeddings (Step {step})')
            plt.tight_layout()
            plt.savefig(f'./learned_embeddings_step_{step}.png', dpi=300, bbox_inches='tight')
            plt.close()
            
            print(f"Learned embeddings visualization saved to './learned_embeddings_step_{step}.png'")
            
        except ImportError:
            print("Matplotlib or sklearn not available for embedding visualization")

class EmbeddingTrainingCallback(BaseCallback):
    """Callback to monitor embedding learning"""
    
    def __init__(self, check_interval=5000, verbose=0):
        super().__init__(verbose)
        self.check_interval = check_interval
        self.embedding_history = {}
    
    def _on_step(self):
        if self.n_calls % self.check_interval == 0:
            # Log embedding statistics
            if hasattr(self.model.policy, 'embedding_manager'):
                embeddings = self.model.policy.embedding_manager.get_all_embeddings()
                
                # Calculate statistics
                embedding_norms = np.linalg.norm(embeddings, axis=1)
                embedding_std = np.std(embeddings, axis=0)
                
                self.logger.record("embeddings/mean_norm", np.mean(embedding_norms))
                self.logger.record("embeddings/std_norm", np.std(embedding_norms))
                self.logger.record("embeddings/max_norm", np.max(embedding_norms))
                self.logger.record("embeddings/min_norm", np.min(embedding_norms))
                
                # Log some individual embedding dimensions
                for i in range(min(4, embeddings.shape[1])):
                    self.logger.record(f"embeddings/dim_{i}_mean", np.mean(embeddings[:, i]))
                    self.logger.record(f"embeddings/dim_{i}_std", np.std(embeddings[:, i]))
                
                # Visualize embeddings every 50k steps
                if self.n_calls % 50000 == 0:
                    self.model.policy.embedding_manager.visualize_embeddings(step=self.n_calls)
                    if mlflow.active_run():
                        mlflow.log_artifact(f'./learned_embeddings_step_{self.n_calls}.png')
        
        return True

class ImprovedTrainingCallback(BaseCallback):
    """Enhanced training callback with adaptive learning rate"""
    def __init__(self, check_interval=1000, verbose=0):
        super().__init__(verbose)
        self.check_interval = check_interval
        self.best_mean_reward = -np.inf
        
    def _on_step(self):
        if self.n_calls % self.check_interval == 0:
            mean_reward, std_reward = evaluate_policy(
                self.model, 
                self.model.get_env(), 
                n_eval_episodes=5,
                deterministic=False
            )
            
            self.logger.record("eval/mean_reward", mean_reward)
            self.logger.record("eval/std_reward", std_reward)
            
            # Adaptive learning rate
            if mean_reward > self.best_mean_reward:
                self.best_mean_reward = mean_reward
            else:
                current_lr = self.model.learning_rate
                new_lr = current_lr * 0.99
                self.model.learning_rate = new_lr
            
            # Early stopping for good performance
            if mean_reward > 200:
                return False
                
        return True

class MLflowGradAndRewardCallback(BaseCallback):
    """Callback para logging en MLflow"""
    def __init__(self, verbose=0):
        super().__init__(verbose)
        
    def _on_step(self):
        return True

class ObjectAwareMlpLstmPolicy(nn.Module):
    """Custom policy that incorporates trainable object embeddings"""
    
    def __init__(self, observation_space, action_space, lr_schedule=None, 
                 net_arch=None, activation_fn=nn.ReLU, 
                 lstm_hidden_size=256, n_lstm_layers=1, 
                 enable_critic_lstm=True, **kwargs):
        super().__init__()
        
        # Extract dimensions
        self.observation_space = observation_space
        self.action_space = action_space
        self.embedding_dim = 16
        
        # Object embedding manager
        self.embedding_manager = TrainableObjectEmbeddingManager(embedding_dim=self.embedding_dim)
        
        # Policy network architecture
        self.net_arch = net_arch or [64, 64]
        self.activation_fn = activation_fn
        
        # Build feature extractor
        self.feature_extractor = self._build_feature_extractor()
        
        # LSTM layer
        self.lstm_hidden_size = lstm_hidden_size
        self.n_lstm_layers = n_lstm_layers
        self.lstm = nn.LSTM(
            input_size=self.net_arch[-1],
            hidden_size=lstm_hidden_size,
            num_layers=n_lstm_layers,
            batch_first=True
        )
        
        # Policy and value heads
        self.policy_net = nn.Linear(lstm_hidden_size, action_space.shape[0])
        self.value_net = nn.Linear(lstm_hidden_size, 1)
        
        # Initialize weights
        self.apply(self._init_weights)
        
        print(f"Initialized ObjectAwareMlpLstmPolicy with {observation_space.shape[0]} input dimensions")
    
    def _build_feature_extractor(self):
        """Build the feature extraction network"""
        layers = []
        prev_size = self.observation_space.shape[0]
        
        for layer_size in self.net_arch:
            layers.append(nn.Linear(prev_size, layer_size))
            layers.append(self.activation_fn())
            prev_size = layer_size
        
        return nn.Sequential(*layers)
    
    def _init_weights(self, module):
        """Initialize weights"""
        if isinstance(module, nn.Linear):
            torch.nn.init.orthogonal_(module.weight, gain=1.0)
            if module.bias is not None:
                module.bias.data.fill_(0.0)
    
    def forward(self, obs, lstm_states=None, episode_starts=None):
        """
        Forward pass - embeddings are already included in observations
        """
        batch_size = obs.shape[0] if len(obs.shape) > 1 else 1
        
        # Ensure obs is 2D
        if len(obs.shape) == 1:
            obs = obs.unsqueeze(0)
        
        # Feature extraction (embeddings already in observations)
        features = self.feature_extractor(obs)
        
        # LSTM processing
        if lstm_states is None:
            lstm_states = self._init_lstm_states(batch_size, obs.device)
        
        # Reshape for LSTM (batch_size, seq_len=1, features)
        features = features.unsqueeze(1)
        lstm_out, lstm_states = self.lstm(features, lstm_states)
        lstm_out = lstm_out.squeeze(1)  # Remove sequence dimension
        
        # Policy and value outputs
        actions = self.policy_net(lstm_out)
        values = self.value_net(lstm_out)
        
        return actions, values, lstm_states
    
    def _init_lstm_states(self, batch_size, device):
        """Initialize LSTM states"""
        h = torch.zeros(self.n_lstm_layers, batch_size, self.lstm_hidden_size, device=device)
        c = torch.zeros(self.n_lstm_layers, batch_size, self.lstm_hidden_size, device=device)
        return (h, c)

class MujocoEnvCNN(gym.Env):
    """Enhanced Mujoco environment for robotic grasping with trainable object embeddings"""
    
    def __init__(self, scene_path=None, movement_pattern='down', obs_shape=(300, 300, 3), num_frames=1000000, cfg=None):
        super().__init__()
        
        # Initialize variables
        self.n_episodes = 0
        self.force_mean = np.zeros(12)
        self.force_std = np.ones(12)
        self.force_count = 0
        self.predicted_movement = None
        self.num_agents = 1
        self.scene_path = "alarmclock_lift_s4_scene.xml"
        self.start_time = time.time()
        self.experiment = 2
        self.alpha = 0.1
        self.contact_frames = 0
        self.contact_frames_upward = 0
        self.obs_shape = obs_shape
        self.not_grabbed = True
        self.total_module = 0
        self.summed_total_module = 0
        self.total_reward = 0
        self.reward_success = cfg['reward_success']
        self.penalty_invalid_point = cfg['penalization_out_of_bounds']
        self.frame_idx = 1
        self.x = 0.005
        self.movement_pattern = movement_pattern
        self.max_dist_tol = 0.11
        self.frame_tol = 30
        self.done = False
        self.frame_counter = 0
        self.num_frames = num_frames
        self.max_hand_height = 0.30
        self.scene_results = {}
        self.contact_history = deque(maxlen=10)
        self.force_history = deque(maxlen=5)
        self.reward_history = deque(maxlen=20)
        self.current_state = 0.0
        self.std_state = 0.0
        self.summary = ""
        self.current_object_name = None
        
        # Trainable embedding manager
        self.embedding_manager = TrainableObjectEmbeddingManager(embedding_dim=16)
        
        # Body parts definition
        self.body_parts = {
            "ff_base": 5, "ff_proximal": 6, "ff_medial": 7, "ff_distal": 8, "ff_tip": 9,
            "mf_base": 10, "mf_proximal": 11, "mf_medial": 12, "mf_distal": 13, "mf_tip": 14,
            "rf_base": 15, "rf_proximal": 16, "rf_medial": 17, "rf_distal": 18, "rf_tip": 19,
            "th_base": 20, "th_proximal": 21, "th_medial": 22, "th_distal": 23, "th_tip": 24
        }

        # Load scene
        gesture, sub, _ = extract_s_pattern(self.scene_path)
        objeto = gesture.split('_')[0]
        self.objeto = objeto
        objeto_stl = check_object[objeto.replace('_', '')]
        model_path = "/home/diego/TFM/models/scenes/" + gesture + "_" + sub + "_scene.xml"
        self.model = mujoco.MjModel.from_xml_path(model_path)
        self.data = mujoco.MjData(self.model)
        
        # Initialize simulation
        mujoco.mj_forward(self.model, self.data)
        for _ in range(20):
            mujoco.mj_forward(self.model, self.data)

        # Joint and body IDs
        hand_joint_names = [
            "ffj0", "ffj1", "ffj2", "ffj3", "mfj0", "mfj1", "mfj2", "mfj3",
            "rfj0", "rfj1", "rfj2", "rfj3", "thj0", "thj1", "thj2", "thj3",
            "supination", "wrist_flexion"
        ]
        
        self.hand_joint_indices = [
            mujoco.mj_name2id(self.model, mujoco.mjtObj.mjOBJ_JOINT, name)
            for name in hand_joint_names
        ]

        self.hand_id = mujoco.mj_name2id(self.model, mujoco.mjtObj.mjOBJ_BODY, "hand_root")
        self.obj_id = mujoco.mj_name2id(self.model, mujoco.mjtObj.mjOBJ_BODY, objeto_stl.replace('.stl', '') + "_mesh")
        self.ff_tip_id = mujoco.mj_name2id(self.model, mujoco.mjtObj.mjOBJ_BODY, "ff_tip")
        self.mf_tip_id = mujoco.mj_name2id(self.model, mujoco.mjtObj.mjOBJ_BODY, "mf_tip")
        self.rf_tip_id = mujoco.mj_name2id(self.model, mujoco.mjtObj.mjOBJ_BODY, "rf_tip")
        self.th_tip_id = mujoco.mj_name2id(self.model, mujoco.mjtObj.mjOBJ_BODY, "th_tip")
        self.mocap_id = mujoco.mj_name2id(self.model, mujoco.mjtObj.mjOBJ_BODY, "hand_root_mocap")

        # Update observation space to include object embedding
        original_obs_dim = 39
        embedding_dim = 16
        total_obs_dim = original_obs_dim + embedding_dim
        
        # Action and observation spaces
        self.action_space = spaces.Box(
            low=np.array([self.model.actuator_ctrlrange[i, 0] for i in range(self.model.nu)], dtype=np.float32),
            high=np.array([self.model.actuator_ctrlrange[i, 1] for i in range(self.model.nu)], dtype=np.float32),
            dtype=np.float32
        )

        self.observation_space = spaces.Box(
            low=-np.inf,
            high=np.inf,
            shape=(total_obs_dim,),  # Updated to include embeddings
            dtype=np.float32
        )

    def contact_hand(self):
        """Check if hand is in contact with object"""
        for i in range(self.data.ncon):
            contact = self.data.contact[i]
            if ((contact.geom1 == self.hand_id and contact.geom2 == self.obj_id) or
                (contact.geom2 == self.hand_id and contact.geom1 == self.obj_id)):
                return True
        return False

    def get_fingertip_forces(self):
        """Get fingertip forces"""
        force_vectors = []
        fingertip_ids = [self.ff_tip_id, self.mf_tip_id, self.rf_tip_id, self.th_tip_id]
        for tip_id in fingertip_ids:
            tip_force = np.zeros(3)
            for i in range(self.data.ncon):
                contact = self.data.contact[i]
                if ((contact.geom1 == tip_id and contact.geom2 == self.obj_id) or
                    (contact.geom2 == tip_id and contact.geom1 == self.obj_id)):
                    force = np.zeros(6)
                    mujoco.mj_contactForce(self.model, self.data, i, force)
                    tip_force += force[:3]
            force_vectors.append(tip_force)
        return np.array(force_vectors)

    def force_z_projection(self):
        """Calculate force projection on Z-axis"""
        fingertip_forces = self.get_fingertip_forces()
        total_force = np.sum(fingertip_forces, axis=0)
        z_projection = np.linalg.norm(total_force[2]) / (np.linalg.norm(total_force) + 1e-6)
        return z_projection

    def fingertip_contact(self):
        """Check fingertip contacts with object"""
        fingertip_ids = [self.ff_tip_id, self.mf_tip_id, self.rf_tip_id, self.th_tip_id]
        contact_dict = {tip_id: False for tip_id in fingertip_ids}
        for tip_id in fingertip_ids:
            for i in range(self.data.ncon):
                contact = self.data.contact[i]
                if ((contact.geom1 == tip_id and contact.geom2 == self.obj_id) or
                    (contact.geom2 == tip_id and contact.geom1 == self.obj_id)):
                    contact_dict[tip_id] = True
                    break
        return contact_dict

    def reset(self, seed=None):
        """Enhanced reset with intelligent sampling and object tracking"""
        super().reset(seed=seed)
        self.n_episodes += 1
        self.total_reward = 0
        self.contact_frames = 0
        self.contact_frames_upward = 0
        self.start_time = time.time()
        self.final_reward = 0
        self.grasp_success = False
        self.total_module = 0
        self.summed_total_module = 0
        self.done = False
        self.frame_counter = 0
        self.movement_pattern = 'down'
        
        # Intelligent scene sampling
        scenes_dir = "/home/diego/TFM/models/scenes/"
        all_scenes = os.listdir(scenes_dir)
        
        # Sample based on previous performance
        if hasattr(self, 'scene_results') and len(self.scene_results) > 10:
            scene_scores = list(self.scene_results.items())
            scene_scores.sort(key=lambda x: x[1])
            start_idx = len(scene_scores) // 4
            end_idx = 3 * len(scene_scores) // 4
            medium_difficulty = scene_scores[start_idx:end_idx]
            
            if medium_difficulty:
                chosen_object = random.choice(medium_difficulty)[0]
                matching_scenes = [s for s in all_scenes if chosen_object in s]
                if matching_scenes:
                    sampled_scene = random.choice(matching_scenes)
                else:
                    sampled_scene = random.choice(all_scenes)
            else:
                sampled_scene = random.choice(all_scenes)
        else:
            sampled_scene = random.choice(all_scenes)

        # Load the sampled scene
        print(sampled_scene)
        gesture, sub, _ = extract_s_pattern(sampled_scene)
        print(gesture)
        objeto = gesture.split('_')[0]
        self.objeto = objeto
        self.current_object_name = objeto  # Store for embedding lookup
        
        objeto_stl = check_object[objeto.replace('_', '')]
        model_path = "/home/diego/TFM/models/scenes/" + gesture + "_" + sub + "_scene.xml"
        self.model = mujoco.MjModel.from_xml_path(model_path)
        self.data = mujoco.MjData(self.model)
        
        # Reinitialize body IDs for new scene
        self.hand_id = mujoco.mj_name2id(self.model, mujoco.mjtObj.mjOBJ_BODY, "hand_root")
        self.obj_id = mujoco.mj_name2id(self.model, mujoco.mjtObj.mjOBJ_BODY, objeto_stl.replace('.stl', '') + "_mesh")
        self.mocap_id = mujoco.mj_name2id(self.model, mujoco.mjtObj.mjOBJ_BODY, "hand_root_mocap")

        self.frame_idx = 1
        self.contact_history.clear()
        self.force_history.clear()
        self.reward_history.clear()
        
        info = {'reset': f"resetting on scene {sampled_scene} with object {objeto}"}
        obs = self._get_obs()
        return obs, info

    def step(self, action):
        """Enhanced step function with improved movement logic"""
        done = False
        object_contact = False
        
        # Current positions
        current_hand_pos = self.data.xpos[self.hand_id]
        current_hand_height = current_hand_pos[2]
        current_obj_pos = self.data.xpos[self.obj_id]
        current_obj_height = current_obj_pos[2]
        height_diff = abs(current_hand_height - current_obj_height)

        # Movement logic
        if self.movement_pattern == 'down':
            if height_diff > self.max_dist_tol:
                multiplier = (height_diff - self.max_dist_tol) ** 0.3
                if multiplier < 0:
                    multiplier = 0
                self.data.mocap_pos[self.mocap_id] -= [0, 0, self.x * multiplier]
            
            if self.contact_hand() or height_diff < self.max_dist_tol:
                self.movement_pattern = 'stand'

        elif self.movement_pattern == 'stand':
            self.frame_counter += 1
            if height_diff > self.max_dist_tol:
                self.movement_pattern = 'down'
            if self.frame_counter > self.frame_tol:
                self.movement_pattern = 'up'

        elif self.movement_pattern == 'up':
            self.grasp_success = self.get_grasp_success()
            if current_hand_height > self.max_hand_height:
                self.done = True
                if self.grasp_success:
                    self.final_reward = 1.0
                    print(f"[{self.objeto}] SUCCESS")
                else:
                    contact_ratio = self.contact_frames / max(self.frame_idx, 1)
                    self.final_reward = contact_ratio
                    print(f"[{self.objeto}] FAILURE - Contact ratio: {contact_ratio:.2f}")
                
                self.scene_results[self.objeto] = self.final_reward
            
            self.data.mocap_pos[self.mocap_id] += [0, 0, self.x]

        # Apply control actions
        kp = 50.0
        kv = 5.0
        for i in range(len(self.hand_joint_indices)):
            if i >= 16:
                kp = 20.0
                kv = 2.0
                self.data.ctrl[i] = action[i]
                continue
            error = action[i] - self.data.qpos[i]
            self.data.ctrl[i] = kp * error + kv * self.data.qvel[i]

        # Calculate reward
        reward = self.reward()

        # Step simulation
        mujoco.mj_step(self.model, self.data)
        self.frame_idx += 1

        if self.grasp_success:
            self.contact_frames += 1

        obs = self._get_obs()
        truncated = False
        info = {}

        return obs, reward, self.done, truncated, info

    def reward(self):
        """Enhanced reward function"""
        reward = 0
        
        # Contact rewards
        contact_dict = self.fingertip_contact()
        thumb_contact = contact_dict[self.th_tip_id]
        other_contacts = sum([contact_dict[self.ff_tip_id], 
                             contact_dict[self.mf_tip_id], 
                             contact_dict[self.rf_tip_id]])
        
        # Base contact reward
        if thumb_contact and other_contacts >= 1:
            reward += 0.1 * other_contacts
        
        # Force projection reward
        force_projection = self.force_z_projection()
        if force_projection > 0.1:
            reward += 0.5 * force_projection
        
        # Grasp stability reward
        if self.grasp_success:
            reward += 0.2
            if self.movement_pattern == 'up':
                reward += 0.1
        
        # Height reward during lifting
        if self.movement_pattern == 'up' and self.grasp_success:
            current_height = self.data.xpos[self.hand_id][2]
            height_reward = (current_height / self.max_hand_height) * 0.3
            reward += height_reward
        
        # Penalties
        joint_velocity_penalty = np.mean(np.abs(self.data.qvel[:15]))
        reward -= 0.01 * joint_velocity_penalty
        
        time_penalty = -0.001 * self.frame_idx
        reward += time_penalty
        
        self.total_reward += reward
        return reward

    def _get_obs(self):
        """Get observation vector with object embedding"""
        # Get forces
        forces = self.get_fingertip_forces().flatten()
        
        # Update normalization
        self._update_force_stats(forces)
        normalized_forces = (forces - self.force_mean) / (self.force_std + 1e-8)
        
        # Get positions and orientations
        fingertip_positions = []
        for tip_id in [self.ff_tip_id, self.mf_tip_id, self.rf_tip_id, self.th_tip_id]:
            fingertip_positions.append(self.data.xpos[tip_id])
        fingertip_positions = np.array(fingertip_positions).flatten()
        
        object_position = self.data.xpos[self.obj_id]
        target_orientation = self.data.xquat[self.obj_id]
        hand_orientation = self.data.xquat[self.hand_id]
        hand_to_object_orientation = target_orientation - hand_orientation
        
        # Get object embedding
        if self.current_object_name:
            object_embedding = self.embedding_manager.get_embedding(self.current_object_name)
        else:
            object_embedding = np.zeros(16)
        
        # Combine observations with object embedding
        combined = np.concatenate([
            normalized_forces, 
            fingertip_positions, 
            object_position, 
            target_orientation, 
            hand_orientation, 
            hand_to_object_orientation,
            object_embedding  # Add object embedding
        ])
        
        return combined.astype(np.float32)

    def _update_force_stats(self, forces):
        """Update force normalization statistics"""
        self.force_count += 1
        delta = forces - self.force_mean
        self.force_mean += delta / self.force_count
        delta2 = forces - self.force_mean
        self.force_std += delta * delta2

    def get_grasp_success(self):
        """Check if grasp is successful"""
        return self._check_contact()

    def _check_contact(self):
        """Check contact between fingers and object"""
        thumb_flag = False
        other_finger_flag = False
        
        for i in range(self.data.ncon):
            contact = self.data.contact[i]
            if (contact.geom1 == self.obj_id or contact.geom2 == self.obj_id):
                other_geom = contact.geom2 if contact.geom1 == self.obj_id else contact.geom1
                geom_name = mujoco.mj_id2name(self.model, mujoco.mjtObj.mjOBJ_GEOM, other_geom)
                if geom_name:
                    if 'th' in geom_name:
                        thumb_flag = True
                    elif any(finger in geom_name for finger in ['ff', 'mf', 'rf']):
                        other_finger_flag = True
        
        return thumb_flag and other_finger_flag

    def render(self, options=None):
        """Render environment"""
        panoramic_view = get_camera_image(self.model, self.data, 'panoramic_view', 
                                        width=self.obs_shape[0], height=self.obs_shape[1], 
                                        options=options)
        panoramic_view_bgr = cv2.cvtColor(panoramic_view, cv2.COLOR_RGB2BGR)
        return None, panoramic_view_bgr

    def check_current_state(self):
        """Check and log current training state"""
        summary = "[FINAL] === Current State Results ===\n"
        for scene, reward in self.scene_results.items():
            summary += f"{scene}: {reward:.2f}\n"
        
        if self.scene_results:
            self.current_state = np.mean(list(self.scene_results.values()))
            self.std_state = np.std(list(self.scene_results.values()))
        else:
            self.current_state = 0
            self.std_state = 0
            
        summary += f"Average reward: {self.current_state:.2f}\n"
        summary += f"Std reward: {self.std_state:.2f}\n"
        self.summary = summary
        print(summary)

    def get_embedding_manager(self):
        """Get the embedding manager for the policy"""
        return self.embedding_manager

def comprehensive_evaluation(model, env, n_episodes=20):
    """Comprehensive model evaluation"""
    results = {
        'success_rate': 0,
        'avg_reward': 0,
        'avg_steps': 0,
        'object_performance': {},
        'contact_quality': 0
    }
    
    successful_episodes = 0
    total_reward = 0
    total_steps = 0
    total_contact_quality = 0
    
    for episode in range(n_episodes):
        obs, info = env.reset()
        episode_reward = 0
        steps = 0
        max_contact_quality = 0
        
        while True:
            action, _ = model.predict(obs, deterministic=True)
            obs, reward, done, truncated, info = env.step(action)
            
            episode_reward += reward
            steps += 1
            
            current_quality = env.force_z_projection() * sum(env.fingertip_contact().values())
            max_contact_quality = max(max_contact_quality, current_quality)
            
            if done or truncated:
                break
        
        total_reward += episode_reward
        total_steps += steps
        total_contact_quality += max_contact_quality
        
        if env.final_reward > 0.8:
            successful_episodes += 1
        
        obj_name = env.objeto
        if obj_name not in results['object_performance']:
            results['object_performance'][obj_name] = []
        results['object_performance'][obj_name].append(episode_reward)
    
    results['success_rate'] = successful_episodes / n_episodes
    results['avg_reward'] = total_reward / n_episodes
    results['avg_steps'] = total_steps / n_episodes
    results['contact_quality'] = total_contact_quality / n_episodes
    
    return results

def setup_adaptive_training():
    """Setup adaptive training configuration"""
    config = {
        'initial_lr': 3e-4,
        'min_lr': 1e-6,
        'lr_decay': 0.995,
        'clip_range': 0.2,
        'ent_coef': 0.01,
        'batch_size': 128,
        'n_steps': 2048
    }
    
    # Load training history if exists
    if os.path.exists("./training_history.json"):
        with open("./training_history.json", "r") as f:
            history = json.load(f)
            if history.get('best_reward', 0) > 100:
                config['ent_coef'] = 0.001
                config['lr_decay'] = 0.999
    
    return config

# Main execution
if __name__ == "__main__":
    # Load configuration
    with open('config.yaml', 'r') as f:
        cfg = yaml.safe_load(f)

    # Object database
    stl_objects = [
        "airplane.stl", "alarmclock.stl", "apple.stl", "banana.stl", "binoculars.stl",
        "bowl.stl", "camera.stl", "phone.stl", "cubelarge.stl", "cubemedium.stl",
        "cubesmall.stl", "cup.stl", "cylinderlarge.stl", "cylindermedium.stl",
        "cylindersmall.stl", "doorknob.stl", "elephant.stl", "eyeglasses.stl",
        "flashlight.stl", "flute.stl", "hammer.stl", "headphones.stl",
        "knife.stl", "lightbulb.stl", "mouse.stl", "mug.stl", "pan.stl", "piggybank.stl",
        "pscontroller.stl", "pyramidlarge.stl", "pyramidmedium.stl", "pyramidsmall.stl",
        "rubberduck.stl", "scissors.stl", "spherelarge.stl", "spheremedium.stl",
        "spheresmall.stl", "stanfordbunny.stl", "stapler.stl", "toothbrush.stl",
        "toothpaste.stl", "toruslarge.stl", "torusmedium.stl", "torussmall.stl",
        "train.stl", "utahteapot.stl", "waterbottle.stl", "wineglass.stl", "wristwatch.stl",
    ]

    check_object = {}
    for obj in stl_objects:
        base_name = obj.replace('.stl', '')
        check_object[base_name] = obj

    # Create directories
    os.makedirs("./checkpoints_rppo", exist_ok=True)
    os.makedirs("./videos_rppo", exist_ok=True)
    os.makedirs("./model_checkpoints", exist_ok=True)

    # Create environment with trainable embeddings
    env = MujocoEnvCNN(cfg=cfg)
    check_env(env)

    # Setup callbacks
    callback = MLflowGradAndRewardCallback()
    improved_callback = ImprovedTrainingCallback()
    embedding_callback = EmbeddingTrainingCallback()

    # Try to load existing model
    checkpoint_dir = "./checkpoints_rppo"
    pattern = re.compile(r"rppo_model_(\d+)_reward_([-\d\.]+)\.zip")
    checkpoints = []
    
    for fname in os.listdir(checkpoint_dir):
        match = pattern.match(fname)
        if match:
            timesteps = int(match.group(1))
            reward = float(match.group(2))
            checkpoints.append((fname, timesteps, reward))

    if checkpoints:
        best_checkpoint = max(checkpoints, key=lambda x: x[2])
        best_fname, best_timesteps, best_reward = best_checkpoint
        best_model_path = os.path.join(checkpoint_dir, best_fname)
        
        try:
            # Try to load the model with the new environment
            model = RecurrentPPO.load(best_model_path, env=env)
            print(f"Loaded best model: {best_fname} with reward {best_reward}")
            current_timesteps = best_timesteps
            
            # Transfer embedding manager to policy if it exists
            if hasattr(env, 'embedding_manager') and not hasattr(model.policy, 'embedding_manager'):
                model.policy.embedding_manager = env.embedding_manager
                print("Transferred embedding manager from environment to policy")
                
        except (ValueError, RuntimeError) as e:
            if "Observation spaces do not match" in str(e) or "size mismatch" in str(e):
                print(f"Model incompatible with new observation space. Starting fresh training.")
                print(f"Reason: {e}")
                model = None
                current_timesteps = 0
            else:
                raise e
    else:
        print("No checkpoints found. Starting training from scratch.")
        model = None
        current_timesteps = 0

    if model is None:
        print("Starting training from scratch with trainable object embeddings")
        current_timesteps = 0
        
        # Enhanced policy configuration
        policy_kwargs = dict(
            net_arch=dict(
                pi=[512, 256, 128],
                vf=[512, 256, 128]
            ),
            activation_fn=nn.ReLU,
            lstm_hidden_size=512,
            n_lstm_layers=2,
            enable_critic_lstm=True,
            ortho_init=False,
            share_features_extractor=False
        )

        # Create model
        model = RecurrentPPO(
            "MlpLstmPolicy",
            env,
            learning_rate=3e-4,
            n_steps=2048,
            batch_size=128,
            n_epochs=10,
            gamma=0.99,
            gae_lambda=0.95,
            clip_range=0.2,
            clip_range_vf=0.2,
            ent_coef=0.01,
            vf_coef=0.5,
            max_grad_norm=0.8,
            tensorboard_log="./rppo_tensorboard_embeddings/",
            device='cuda' if torch.cuda.is_available() else 'cpu',
            policy_kwargs=policy_kwargs,
            verbose=1
        )
        
        # Transfer embedding manager to policy
        model.policy.embedding_manager = env.embedding_manager
        print("Embedding manager transferred to policy for training")

    # Training phases
    training_phases = [
        {"timesteps": 100000, "max_hand_height": 0.3, "objects": "simple"},
        {"timesteps": 200000, "max_hand_height": 0.4, "objects": "medium"},
        {"timesteps": 300000, "max_hand_height": 0.5, "objects": "complex"}
    ]

    # Simple objects for phased training
    simple_objects = ["cubesmall", "spheresmall", "cylindersmall", "cubemedium"]
    medium_objects = ["spheremedium", "cylindermedium", "apple", "banana"]
    
    # Start MLflow
    mlflow.start_run()
    
    # Training loop
    total_timesteps = 1000000
    checkpoint_interval = 5000
    
    current_phase = 0
    total_trained = current_timesteps

    while total_trained < total_timesteps and current_phase < len(training_phases):
        phase = training_phases[current_phase]
        print(f"Starting phase {current_phase + 1}: {phase}")
        
        # Configure environment for current phase
        env.max_hand_height = phase["max_hand_height"]
        
        if phase["objects"] == "simple":
            env.allowed_objects = simple_objects
        elif phase["objects"] == "medium":
            env.allowed_objects = medium_objects
        else:
            env.allowed_objects = None

        # Train in current phase
        phase_timesteps = min(phase["timesteps"], total_timesteps - total_trained)
        
        send_text(f"Starting phase {current_phase + 1} with {phase_timesteps} timesteps")
        
        model.learn(
            total_timesteps=phase_timesteps,
            callback=[callback, improved_callback, embedding_callback],
            reset_num_timesteps=False
        )
        
        total_trained += phase_timesteps
        
        # Evaluate and save
        mean_reward, std_reward = evaluate_policy(model, env, n_eval_episodes=5)
        env.check_current_state()
        
        # Save checkpoint
        model.save(f"./checkpoints_rppo/phase_{current_phase+1}_{total_trained}_reward_{mean_reward:.2f}.zip")
        
        # Record video
        record_final_video(model, env, f"./videos_rppo/phase_{current_phase+1}_{total_trained}.mp4")
        
        # Adaptive difficulty
        if env.current_state > 0.85 and current_phase < len(training_phases) - 1:
            print(f"Progressing to next phase. Current success rate: {env.current_state:.2f}")
            current_phase += 1
        elif env.current_state > 0.85:
            env.max_hand_height += 0.02
            print(f"Increasing difficulty. New max height: {env.max_hand_height}")
        
        send_text(f"Phase {current_phase + 1} completed. Mean reward: {mean_reward:.2f}")
        send_text(env.summary)

    # Final evaluation
    print("Training completed. Running final evaluation...")
    final_results = comprehensive_evaluation(model, env, n_episodes=20)
    print("Final Results:", final_results)
    
    # Record final video
    record_final_video(model, env, "./videos_rppo/final_agent.mp4")
    
    # Save final model
    model.save("final_grasping_policy_rppo_with_embeddings")
    
    # Visualize final embeddings
    if hasattr(model.policy, 'embedding_manager'):
        model.policy.embedding_manager.visualize_embeddings(step=total_timesteps)
        mlflow.log_artifact(f'./learned_embeddings_step_{total_timesteps}.png')
    
    mlflow.end_run()
    print("Training completed successfully!")

Initialized trainable embeddings for 50 objects
Objects: ['airplane', 'alarmclock', 'apple', 'banana', 'binoculars', 'bowl', 'camera', 'cubelarge', 'cubemedium', 'cubesmall', 'cup', 'cylinderlarge', 'cylindermedium', 'cylindersmall', 'doorknob', 'duck', 'elephant', 'eyeglasses', 'flashlight', 'flute', 'fryingpan', 'gamecontroller', 'hammer', 'hand', 'headphones', 'knife', 'lightbulb', 'mouse', 'mug', 'phone', 'piggybank', 'pyramidlarge', 'pyramidmedium', 'pyramidsmall', 'scissors', 'spherelarge', 'spheremedium', 'spheresmall', 'stamp', 'stanfordbunny', 'stapler', 'toothbrush', 'toothpaste', 'toruslarge', 'torusmedium', 'torussmall', 'train', 'watch', 'waterbottle', 'wineglass']
torussmall_inspect_1_s9_scene.xml
torussmall_inspect_1
airplane_fly_1_s6_scene.xml
airplane_fly_1
waterbottle_pour_2_s3_scene.xml
waterbottle_pour_2
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.
Model incompatible with new observation space. Starting fresh training.
Reason: Observa

[bowl] FAILURE - Contact ratio: 0.01
bowl_pass_1_s5_scene.xml
bowl_pass_1
[bowl] FAILURE - Contact ratio: 0.00
toothbrush_brush_1_s8_scene.xml
toothbrush_brush_1
[toothbrush] FAILURE - Contact ratio: 0.00
camera_browse_1_s9_scene.xml
camera_browse_1
[camera] FAILURE - Contact ratio: 0.00
knife_chop_1_s7_scene.xml
knife_chop_1
[knife] FAILURE - Contact ratio: 0.00
cylindersmall_inspect_1_s5_scene.xml
cylindersmall_inspect_1
apple_eat_1_s1_scene.xml
apple_eat_1
[apple] FAILURE - Contact ratio: 0.00
bowl_drink_1_s7_scene.xml
bowl_drink_1
[bowl] FAILURE - Contact ratio: 0.03
apple_eat_1_s4_scene.xml
apple_eat_1
[apple] FAILURE - Contact ratio: 0.00
train_pass_1_s2_scene.xml
train_pass_1
[train] FAILURE - Contact ratio: 0.00
apple_pass_1_s3_scene.xml
apple_pass_1
[apple] FAILURE - Contact ratio: 0.00
train_pass_1_s4_scene.xml
train_pass_1
[train] FAILURE - Contact ratio: 0.00
cylindersmall_inspect_1_s10_scene.xml
cylindersmall_inspect_1
cylindersmall_pass_1_s5_scene.xml
cylindersmall_pass_1

[train] FAILURE - Contact ratio: 0.00
train_play_1_s2_scene.xml
train_play_1
[train] FAILURE - Contact ratio: 0.00
camera_lift_s4_scene.xml
camera_lift
[camera] FAILURE - Contact ratio: 0.00
train_offhand_1_s1_scene.xml
train_offhand_1
[train] FAILURE - Contact ratio: 0.00
knife_peel_1_s6_scene.xml
knife_peel_1
cylindersmall_lift_s8_scene.xml
cylindersmall_lift
[cylindersmall] FAILURE - Contact ratio: 0.00
cylindersmall_pass_1_s6_scene.xml
cylindersmall_pass_1
[cylindersmall] FAILURE - Contact ratio: 0.00
train_pass_1_s3_scene.xml
train_pass_1
[train] FAILURE - Contact ratio: 0.00
cylindersmall_pass_1_s5_scene.xml
cylindersmall_pass_1
[cylindersmall] FAILURE - Contact ratio: 0.00
toothbrush_brush_1_s5_scene.xml
toothbrush_brush_1
[toothbrush] FAILURE - Contact ratio: 0.00
toothbrush_brush_1_s8_scene.xml
toothbrush_brush_1
[toothbrush] FAILURE - Contact ratio: 0.00
camera_takepicture_2_s6_scene.xml
camera_takepicture_2
[camera] FAILURE - Contact ratio: 0.00
train_pass_1_s2_scene.xml
tra

[camera] FAILURE - Contact ratio: 0.01
camera_pass_1_s4_scene.xml
camera_pass_1
[camera] FAILURE - Contact ratio: 0.00
camera_pass_1_s6_scene.xml
camera_pass_1
-----------------------------------------
| eval/                   |             |
|    mean_reward          | -9.86e+04   |
|    std_reward           | 1.97e+05    |
| rollout/                |             |
|    ep_len_mean          | 128         |
|    ep_rew_mean          | -9.7        |
| time/                   |             |
|    fps                  | 12          |
|    iterations           | 9           |
|    time_elapsed         | 1464        |
|    total_timesteps      | 18432       |
| train/                  |             |
|    approx_kl            | 0.012516878 |
|    clip_fraction        | 0.0901      |
|    clip_range           | 0.2         |
|    clip_range_vf        | 0.2         |
|    entropy_loss         | -25.9       |
|    explained_variance   | 0.0733      |
|    learning_rate        | 0.0003      |


[apple] FAILURE - Contact ratio: 0.00
elephant_pass_1_s2_scene.xml
elephant_pass_1
[elephant] FAILURE - Contact ratio: 0.04
cylindersmall_inspect_1_s3_scene.xml
cylindersmall_inspect_1
-----------------------------------------
| eval/                   |             |
|    mean_reward          | -2.95e+05   |
|    std_reward           | 2.41e+05    |
| rollout/                |             |
|    ep_len_mean          | 128         |
|    ep_rew_mean          | -9.63       |
| time/                   |             |
|    fps                  | 12          |
|    iterations           | 12          |
|    time_elapsed         | 1980        |
|    total_timesteps      | 24576       |
| train/                  |             |
|    approx_kl            | 0.006440622 |
|    clip_fraction        | 0.0979      |
|    clip_range           | 0.2         |
|    clip_range_vf        | 0.2         |
|    entropy_loss         | -26.1       |
|    explained_variance   | 0.177       |
|    learning_rat

knife_lift_s6_scene.xml
knife_lift
[knife] FAILURE - Contact ratio: 0.00
knife_pass_1_s9_scene.xml
knife_pass_1
[knife] FAILURE - Contact ratio: 0.00
train_lift_s4_scene.xml
train_lift
[train] FAILURE - Contact ratio: 0.04
cylindersmall_inspect_1_s7_scene.xml
cylindersmall_inspect_1
[cylindersmall] FAILURE - Contact ratio: 0.00
toothbrush_pass_1_s4_scene.xml
toothbrush_pass_1
[toothbrush] FAILURE - Contact ratio: 0.00
knife_lift_s6_scene.xml
knife_lift
[knife] FAILURE - Contact ratio: 0.00
toothbrush_brush_1_s3_scene.xml
toothbrush_brush_1
[toothbrush] FAILURE - Contact ratio: 0.00
apple_lift_s8_scene.xml
apple_lift
[apple] FAILURE - Contact ratio: 0.00
knife_lift_s3_scene.xml
knife_lift
[knife] FAILURE - Contact ratio: 0.00
apple_lift_s1_scene.xml
apple_lift
[apple] FAILURE - Contact ratio: 0.00
camera_offhand_1_s3_scene.xml
camera_offhand_1
[camera] FAILURE - Contact ratio: 0.00
cup_pass_1_s6_scene.xml
cup_pass_1
[cup] FAILURE - Contact ratio: 0.00
knife_lift_s6_scene.xml
knife_lift


ValueError: XML Error: Error opening file '/home/diego/TFM/models/objects/cup_lift_object_s1.xml': No such file or directory
Element 'include', line 3


In [18]:
mlflow.end_run()